In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [4]:
listing = pd.read_csv('bt5153_gp-main/listings.csv')
listing.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56861,-0.11270,Private room,65,1,22,2021-11-21,0.16,2,361,1,NaN
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.48780,-0.16813,Entire home/apt,75,3,89,2020-03-16,0.61,1,4,0,NaN
2,17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Liz,NaN,Westminster,51.52195,-0.14094,Entire home/apt,265,4,43,2021-06-28,0.33,1,295,1,NaN
3,17506,Boutique Chelsea/Fulham Double bed 5-star ensuite,67915,Charlotte,NaN,Hammersmith and Fulham,51.47935,-0.19743,Private room,150,3,0,NaN,NaN,2,364,0,NaN
4,25123,Clean big Room in London (Room 1),103583,Grace,NaN,Barnet,51.57438,-0.21081,Private room,29,10,129,2020-03-15,0.91,3,0,0,NaN


In [5]:
station = pd.read_csv('london_stations.csv')
station.head()

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508757,-0.263430,2,W3 6BH
3,Acton Main Line,520296,181196,51.516886,-0.267690,3,W3 9EH
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN


In [9]:
for listing_row_id, listing_row in listing.iterrows():
    min_dist_station = 9999999
    min_station = ""
    count = 0
    origin = list(listing_row[['latitude','longitude']])
    for station_row_id, station_row in station.iterrows():
        dest = list(station_row[['Latitude','Longitude']])
        dist = geodesic(origin,dest).meters
        if dist<min_dist_station:
            min_dist_station = dist
            min_station = station.iloc[station_row_id][0]
        if dist<800:
            count = count + 1
    listing.loc[[listing_row_id],['station']] = min_station
    listing.loc[[listing_row_id],['station_dist']] = min_dist_station
    listing.loc[[listing_row_id],['walking_distance']] = count
listing.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,station,station_dist,walking_distance
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56861,-0.11270,Private room,65,...,22,2021-11-21,0.16,2,361,1,NaN,Crouch Hill,431.108995,2.0
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.48780,-0.16813,Entire home/apt,75,...,89,2020-03-16,0.61,1,4,0,NaN,South Kensington,805.406363,0.0
2,17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Liz,NaN,Westminster,51.52195,-0.14094,Entire home/apt,265,...,43,2021-06-28,0.33,1,295,1,NaN,Great Portland Street,292.948350,5.0
3,17506,Boutique Chelsea/Fulham Double bed 5-star ensuite,67915,Charlotte,NaN,Hammersmith and Fulham,51.47935,-0.19743,Private room,150,...,0,NaN,NaN,2,364,0,NaN,Fulham Broadway,165.612825,2.0
4,25123,Clean big Room in London (Room 1),103583,Grace,NaN,Barnet,51.57438,-0.21081,Private room,29,...,129,2020-03-15,0.91,3,0,0,NaN,Brent Cross,302.665825,1.0


In [10]:
listing.to_csv(r'london_nearest_tube.csv', index = False, header=True)

In [ ]:
#station: nearest station from listing
#station_dist: distance to nearest station from listing
#walking_distance: how many stations are in 10 mins walking distance (assumption: walking speed 1.33 m/s)